In [2]:
import pandas as pd
import csv
import json 
from collections import ChainMap
from nltk.stem import WordNetLemmatizer
import random
import contractions

In [3]:
df = pd.read_csv("train", sep='\t', quoting=csv.QUOTE_NONE, header=None)
len(df)
df.loc[0] = [1, ".", "."]
#df[0:30]

In [4]:
df.columns =['Index', 'Name', 'POS']

In [5]:
from collections import Counter
results = Counter()
df['Name'].str.lower().str.split().apply(results.update)

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
912065    None
912066    None
912067    None
912068    None
912069    None
912070    None
912071    None
912072    None
912073    None
912074    None
912075    None
912076    None
912077    None
912078    None
912079    None
912080    None
912081    None
912082    None
912083    None
912084    None
912085    None
912086    None
912087    None
912088    None
912089    None
912090    None
912091    None
912092    None
912093    None
912094    None
Name: Name, Length: 912095, dtype: object

In [6]:
print(df['Name'])

0                    .
1               Vinken
2                    ,
3                   61
4                years
5                  old
6                    ,
7                 will
8                 join
9                  the
10               board
11                  as
12                   a
13        nonexecutive
14            director
15                Nov.
16                  29
17                   .
18                 Mr.
19              Vinken
20                  is
21            chairman
22                  of
23            Elsevier
24                N.V.
25                   ,
26                 the
27               Dutch
28          publishing
29               group
              ...     
912065              of
912066          Type-O
912067           blood
912068               .
912069              It
912070              is
912071            also
912072         pulling
912073              20
912074          people
912075             out
912076              of
912077     

In [7]:
for word in list(results):
    #print(results[word])
    if results[word]<2:
        results["<unk>"] += results[word]
        del results[word]

In [8]:
results = dict(results)

In [9]:
sorted_results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse = True)}
print(sorted_results)

{',': 46476, 'the': 46144, '.': 37453, 'of': 22176, 'to': 21459, 'a': 19338, '<unk>': 17401, 'in': 16320, 'and': 15875, "'s": 8886, 'for': 8190, 'that': 8157, 'is': 6764, '$': 6762, '``': 6673, "''": 6500, 'it': 5601, 'said': 5427, 'on': 5120, '%': 4718, 'at': 4460, 'by': 4410, 'as': 4389, 'from': 4279, 'with': 4138, 'million': 4122, 'mr.': 3856, 'are': 3642, 'was': 3619, 'be': 3590, 'its': 3467, 'he': 3310, 'but': 3220, 'has': 3188, 'an': 3156, "n't": 3133, 'have': 3000, 'will': 2993, 'new': 2573, 'or': 2460, 'company': 2456, 'they': 2352, 'this': 2228, 'year': 2175, 'which': 2173, 'would': 2113, 'about': 2045, '--': 2005, 'says': 1978, 'more': 1908, 'market': 1829, 'were': 1779, 'their': 1698, 'billion': 1680, 'his': 1644, 'up': 1635, 'had': 1631, 'one': 1626, 'than': 1589, 'some': 1558, 'u.s.': 1552, 'who': 1539, 'been': 1501, 'also': 1485, 'other': 1438, 'stock': 1426, 'share': 1367, 'not': 1333, ':': 1292, 'we': 1263, 'i': 1189, 'if': 1173, 'corp.': 1166, 'when': 1164, 'last': 116

In [10]:
index=2
with open('vocab.txt', 'a') as vocab_file:
    vocab_file.write("<unk>\t1\t"+str(sorted_results["<unk>"])+"\n")
    for key,value in sorted_results.items():
        vocab_file.write(key+"\t"+str(index)+"\t"+str(value)+"\n")
        index+=1

In [1]:
vocab = set(sorted_results.keys())
print(len(vocab))
print(sorted_results['arizona'])

NameError: name 'sorted_results' is not defined

In [43]:
def replaceWithUnk(word):
    if(word not in vocab):
        return "<unk>"
    else:
        return word

In [44]:
df['Name'] = df['Name'].apply(lambda word : replaceWithUnk(word.lower()))
df['POS'] = df['POS'].apply(lambda word : word.lower())

In [45]:
state_transiton_counts = {}
for i in range(1,len(df['POS'])):
    key = str(df['POS'][i-1])+"^"+str(df['POS'][i])
    if(key not in state_transiton_counts.keys()):
        state_transiton_counts[key] = 1
    else:
        state_transiton_counts[key]+=1

In [46]:
state_transiton_counts

{'.^nnp': 6921,
 'nnp^,': 12131,
 ',^cd': 987,
 'cd^nns': 5502,
 'nns^jj': 1009,
 'jj^,': 1717,
 ',^md': 490,
 'md^vb': 7541,
 'vb^dt': 5662,
 'dt^nn': 37299,
 'nn^in': 31558,
 'in^dt': 31092,
 'dt^jj': 17200,
 'jj^nn': 26472,
 'nn^nnp': 1223,
 'nnp^cd': 1680,
 'cd^.': 2530,
 'nnp^nnp': 33191,
 'nnp^vbz': 3434,
 'vbz^nn': 751,
 'in^nnp': 14095,
 ',^dt': 6213,
 'dt^nnp': 8760,
 'nnp^vbg': 156,
 'vbg^nn': 1819,
 'nn^.': 13890,
 'jj^cc': 1003,
 'cc^jj': 2520,
 ',^vbd': 2396,
 'vbd^vbn': 2696,
 'vbn^dt': 1287,
 'jj^jj': 4362,
 '.^dt': 7844,
 'in^nn': 10363,
 'nn^rb': 2317,
 'rb^vbn': 2389,
 'vbn^to': 2081,
 'to^vb': 12398,
 'vb^nnp': 814,
 'nnp^nn': 5052,
 'nn^nns': 10035,
 'nns^vbz': 493,
 'vbz^vbn': 3093,
 'nns^in': 13571,
 'in^nns': 5674,
 'nns^vbn': 1211,
 'to^prp': 107,
 'prp^rbr': 25,
 'rbr^in': 384,
 'in^cd': 5621,
 'in^,': 239,
 ',^nns': 1271,
 'nns^vbd': 4123,
 'vbd^.': 1308,
 'nn^nn': 15583,
 'nn^,': 14485,
 ',^nn': 2283,
 ',^vbz': 1456,
 'vbz^rb': 2827,
 'rb^jj': 3031,
 'jj^in':

In [47]:
state_counts = {}
for i in range(0,len(df['POS'])):
    key = str(df['POS'][i])
    if(key not in state_counts.keys()):
        state_counts[key] = 1
    else:
        state_counts[key]+=1

In [48]:
state_counts

{'.': 37884,
 'nnp': 87607,
 ',': 46480,
 'cd': 34876,
 'nns': 57859,
 'jj': 58944,
 'md': 9437,
 'vb': 25489,
 'dt': 78775,
 'nn': 127534,
 'in': 94758,
 'vbz': 20982,
 'vbg': 14348,
 'cc': 22817,
 'vbd': 28309,
 'vbn': 19330,
 'rb': 29621,
 'to': 21461,
 'prp': 16766,
 'rbr': 1675,
 'wdt': 4194,
 'vbp': 12326,
 'rp': 2515,
 'prp$': 7989,
 'jjs': 1867,
 'pos': 8284,
 '``': 6782,
 'ex': 833,
 "''": 6622,
 'wp': 2285,
 ':': 4680,
 'jjr': 3174,
 'wrb': 2050,
 '$': 6937,
 'nnps': 2505,
 'wp$': 166,
 '-lrb-': 1305,
 '-rrb-': 1321,
 'pdt': 333,
 'rbs': 435,
 'fw': 224,
 'uh': 87,
 'sym': 55,
 'ls': 47,
 '#': 127}

In [49]:
emission_transiton_counts = {}
for i in range(0,len(df['POS'])):
    key = str(df['POS'][i])+"^"+str(df['Name'][i])
    if(key not in emission_transiton_counts.keys()):
        emission_transiton_counts[key] = 1
    else:
        emission_transiton_counts[key]+=1

In [50]:
emission_transiton_counts

{'.^.': 37453,
 'nnp^vinken': 2,
 ',^,': 46476,
 'cd^61': 25,
 'nns^years': 1133,
 'jj^old': 216,
 'md^will': 2970,
 'vb^join': 41,
 'dt^the': 46092,
 'nn^board': 298,
 'in^as': 3667,
 'dt^a': 19266,
 'jj^nonexecutive': 6,
 'nn^director': 309,
 'nnp^nov.': 234,
 'cd^29': 74,
 'nnp^mr.': 3856,
 'vbz^is': 6761,
 'nn^chairman': 432,
 'in^of': 22172,
 'nnp^<unk>': 4000,
 'nnp^n.v.': 13,
 'nnp^dutch': 8,
 'vbg^publishing': 14,
 'nn^group': 603,
 'nnp^rudolph': 8,
 'nnp^agnew': 3,
 'cd^55': 54,
 'cc^and': 15866,
 'jj^former': 263,
 'nnp^consolidated': 14,
 'nnp^gold': 13,
 'nnp^fields': 3,
 'nnp^plc': 105,
 'vbd^was': 3619,
 'vbn^named': 168,
 'dt^this': 2224,
 'jj^british': 194,
 'jj^industrial': 128,
 'nn^conglomerate': 18,
 'nn^form': 82,
 'nn^asbestos': 27,
 'rb^once': 164,
 'vbn^used': 257,
 'to^to': 21456,
 'vb^make': 511,
 'nnp^kent': 11,
 'nn^cigarette': 17,
 'nns^filters': 9,
 'vbz^has': 3186,
 'vbn^caused': 53,
 'jj^high': 308,
 'nn^percentage': 124,
 'nn^cancer': 72,
 'nns^deaths'

In [51]:
transition_probabilites = {}
transition_matrix = ChainMap()
for key in state_transiton_counts.keys():
    backward,forward = key.split("^")
    probability = state_transiton_counts[key] / state_counts[backward]
    transition_key = "("+backward+","+forward+")"
    transition_probabilites[transition_key] = probability
    if backward not in transition_matrix.keys():
        #emisson_matrix[word] = [[state,probability]]
        transition_matrix[backward] = {forward:probability}
    else:
        transition_matrix[backward].update({forward:probability})
        #emisson_matrix[word].append([state,probability])

In [52]:
#print(transition_probabilites)
len(transition_probabilites)

1378

In [53]:
print(transition_matrix['.'])
max(transition_matrix['cd'], key=transition_matrix['cd'].get)

{'nnp': 0.18268926195755464, 'dt': 0.20705310949213387, 'in': 0.12057860838348643, 'prp': 0.05603948896631823, 'ex': 0.003906662443247809, "''": 0.058890296695174746, '``': 0.07380424453595186, 'cd': 0.01037377256889452, 'rbr': 0.0020325203252032522, 'nns': 0.03848590433956288, 'nn': 0.038538697075282444, 'jj': 0.03838031886812374, 'jjr': 0.0016365748073065146, 'rb': 0.05192165558019216, 'wrb': 0.005728011825572801, 'cc': 0.05271354661598564, 'vbg': 0.011033681765389082, 'wdt': 0.0006599091964945623, 'vbn': 0.005490444514834759, '-lrb-': 0.004619364375461937, '-rrb-': 0.005279273571956499, 'vb': 0.0030619786717347693, 'wp': 0.002771618625277162, 'prp$': 0.007390983000739098, 'to': 0.0032203568788934643, 'jjs': 0.0022700876359412945, 'nnps': 0.0020589166930630345, 'vbz': 0.0012934220251293422, 'vbd': 0.0006335128286347799, 'ls': 0.0007654946679336923, ':': 0.002666033153838032, 'vbp': 0.0003431527821771724, 'pdt': 0.0006599091964945623, 'uh': 0.0005807200929152149, 'md': 0.0005015309893

'nn'

In [54]:
emission_probabilites = {}
emisson_matrix = ChainMap()
for key in emission_transiton_counts.keys():
    state,word = key.split("^")
    probability = emission_transiton_counts[key] / state_counts[state]
    transition_key = "("+state+","+word+")"
    emission_probabilites[transition_key] = probability
    if word not in emisson_matrix.keys():
        #emisson_matrix[word] = [[state,probability]]
        emisson_matrix[word] = {state:probability}
    else:
        emisson_matrix[word].update({state:probability})
        #emisson_matrix[word].append([state,probability])

In [55]:
#print(emission_probabilites)
#len(emission_probabilites)
#emission_probabilites['(NNP,Pierre)']
print(emisson_matrix['arizona'])
#max(emisson_matrix['<unk>'], key=emisson_matrix['<unk>'].get)

{'nnp': 0.0006506329402901595}


In [56]:
emisson_matrix["<unk>"]

{'nnp': 0.045658451950186627,
 'nn': 0.02095127573823451,
 'rb': 0.01120826440700854,
 'jj': 0.05985342019543974,
 'nns': 0.026599146200245425,
 'cd': 0.08097258859961004,
 'vbn': 0.025504397309881015,
 'vbd': 0.011233176728248967,
 'vbg': 0.042584332311123504,
 'vb': 0.014712228804582369,
 'jjs': 0.01981788966256026,
 'vbz': 0.0161090458488228,
 'vbp': 0.007301638812266753,
 'fw': 0.19196428571428573,
 'rbr': 0.00417910447761194,
 'nnps': 0.043912175648702596,
 'jjr': 0.018588531821046,
 '$': 0.00014415453366008361,
 '``': 0.0001474491300501327,
 'prp': 0.00023857807467493738,
 'md': 0.0001059658789869662,
 'in': 4.221279469807299e-05,
 'uh': 0.13793103448275862,
 'wdt': 0.0002384358607534573,
 'cc': 4.382697111802603e-05}

In [57]:
with open("hmm.json", "w") as outfile:
    outfile.write("\nTransition Probabilites: \n")
    json.dump(transition_probabilites, outfile)
    outfile.write("\n\nEmission Probabilites: \n")
    json.dump(emission_probabilites, outfile)

In [58]:
with open("dev","r") as devfile:
    corpus = devfile.readlines()
corpus = [sentence.lower() for sentence in corpus]

In [59]:
cleaned_dev_corpus = []
sentence = []
for words in corpus:
    if words[0]!='\n':
        word_tag = words.split("\t")
        word = (word_tag[1],word_tag[2].split('\n')[0])
        if word[0] in vocab:
            sentence.append(word)
        else:
            sentence.append(("<unk>", word[1]))
    else:
        cleaned_dev_corpus.append(sentence)
        sentence = []

In [60]:
len(cleaned_dev_corpus)
cleaned_dev_corpus[0]

[('the', 'dt'),
 ('arizona', 'nnp'),
 ('corporations', 'nnp'),
 ('commission', 'nnp'),
 ('authorized', 'vbd'),
 ('an', 'dt'),
 ('11.5', 'cd'),
 ('%', 'nn'),
 ('rate', 'nn'),
 ('increase', 'nn'),
 ('at', 'in'),
 ('tucson', 'nnp'),
 ('electric', 'nnp'),
 ('power', 'nnp'),
 ('co.', 'nnp'),
 (',', ','),
 ('substantially', 'rb'),
 ('lower', 'jjr'),
 ('than', 'in'),
 ('recommended', 'vbn'),
 ('last', 'jj'),
 ('month', 'nn'),
 ('by', 'in'),
 ('a', 'dt'),
 ('commission', 'nn'),
 ('hearing', 'nn'),
 ('officer', 'nn'),
 ('and', 'cc'),
 ('barely', 'rb'),
 ('half', 'pdt'),
 ('the', 'dt'),
 ('rise', 'nn'),
 ('sought', 'vbn'),
 ('by', 'in'),
 ('the', 'dt'),
 ('utility', 'nn'),
 ('.', '.')]

In [61]:
transition_matrix['.'].items()
print("arizona" in vocab)

True


In [81]:
def GreedyDecoding(sentence):
    Tagged_POS = []
    for i in range(len(sentence)):
        word_to_be_tagged = sentence[i][0]
        if word_to_be_tagged not in vocab:
            word_to_be_tagged = "<unk>"
        #print(word_to_be_tagged)
        max_value = 0
        max_POS = 0
        if(i==0):
            for POS,value in emisson_matrix[word_to_be_tagged].items():
                if POS in transition_matrix['.'].keys():
                    total_probability = value * transition_matrix['.'][POS]
                    if total_probability>=max_value:
                        max_value = total_probability
                        max_POS = POS
            if max_POS == 0:
                max_POS = max(transition_matrix['.'], key=transition_matrix['.'].get)
                #max_POS = random.choice(list(transition_matrix['.'].keys()))
                    
        else:
            for POS,value in emisson_matrix[word_to_be_tagged].items():
                #print(transition_matrix.keys())
                #print(Tagged_POS[i-1])
                if POS in transition_matrix[Tagged_POS[i-1]].keys():
                    total_probability = value * transition_matrix[Tagged_POS[i-1]][POS]
                    if total_probability>=max_value:
                        max_value = total_probability
                        max_POS = POS
            if max_POS == 0:
                max_POS = max(transition_matrix[Tagged_POS[i-1]], key=transition_matrix[Tagged_POS[i-1]].get)
                #max_POS = random.choice(list(transition_matrix[Tagged_POS[i-1]].keys()))
        Tagged_POS.append(max_POS)
        #print(Tagged_POS)
    return(Tagged_POS)
            

In [63]:
def accuracy_model(data, algo):
    correct = 0
    total = 0
    for sentence in data:
        Tagged_POS = algo(sentence)
        Actual_POS = [word[1] for word in sentence]
        correct_list = [1 if Tagged_POS[i]==Actual_POS[i] else 0 for i in range(len(Tagged_POS))]
        sum(correct_list)
        correct += sum(correct_list)
        total += len(correct_list)
    acc = correct/total
    acc = acc * 100
    return acc

In [64]:
transition_matrix['pos']

{'nnp': 0.10598744567841623,
 'nns': 0.13025108643167552,
 '.': 0.005432158377595364,
 'nn': 0.41344760985031387,
 'cd': 0.024867213906325448,
 'vbg': 0.011830033800096573,
 'jj': 0.210164171897634,
 'vbn': 0.00760502172863351,
 'jjs': 0.025470787059391597,
 '``': 0.010622887493964268,
 'wp': 0.00012071463061323032,
 '-rrb-': 0.0009657170449058426,
 'vbz': 0.0035007242877836795,
 ',': 0.005432158377595364,
 'cc': 0.0063978754225012075,
 'vbd': 0.006518590053114438,
 'nnps': 0.00386286817962337,
 'jjr': 0.0022935779816513763,
 'in': 0.0028971511347175277,
 '$': 0.010502172863351039,
 'rbs': 0.0021728633510381457,
 'rb': 0.005432158377595364,
 "''": 0.00036214389183969096,
 'wrb': 0.00012071463061323032,
 'rbr': 0.00036214389183969096,
 'md': 0.0007242877836793819,
 'vbp': 0.00036214389183969096,
 'prp$': 0.00024142926122646064,
 'vb': 0.00036214389183969096,
 'dt': 0.00024142926122646064,
 'fw': 0.00024142926122646064,
 '#': 0.0006035731530661516,
 'prp': 0.00012071463061323032,
 '-lrb-

In [78]:
print(cleaned_dev_corpus[66])
print(GreedyDecoding(cleaned_dev_corpus[66]))

[('the', 'dt'), ('sale', 'nn'), ('price', 'nn'), ('of', 'in'), ('mccall', 'nnp'), ("'s", 'pos'), ('--', ':'), ('twice', 'rb'), ('what', 'wp'), ('mr.', 'nnp'), ('lang', 'nnp'), ('originally', 'rb'), ('paid', 'vbd'), ('for', 'in'), ('it', 'prp'), ('--', ':'), ('will', 'md'), ('finance', 'vb'), ('lang', 'nnp'), ('communications', 'nnp'), ("'", 'pos'), ('buy-back', 'nn'), ('of', 'in'), ('time', 'nnp'), ('warner', 'nnp'), ("'s", 'pos'), ('50', 'cd'), ('%', 'nn'), ('interest', 'nn'), ('in', 'in'), ('working', 'nnp'), ('woman', 'nnp'), ('and', 'cc'), ('working', 'nnp'), ('mother', 'nnp'), ('.', '.')]
['dt', 'nn', 'nn', 'in', 'nnp', 'pos', 'nn', 'rb', 'wp', 'nnp', 'nnp', 'rb', 'vbn', 'in', 'prp', ':', 'md', 'vb', 'nnp', 'nnps', 'pos', 'nn', 'in', 'nn', 'nnp', 'pos', 'cd', 'nn', 'nn', 'in', 'vbg', 'nn', 'cc', 'vbg', 'nn', '.']


In [82]:
accuracy_model(cleaned_dev_corpus, GreedyDecoding)

92.01220484094998

In [67]:
emisson_matrix['the']

{'dt': 0.5851094890510949,
 'nnp': 0.00046799913248941297,
 'jj': 0.00011875678610206297,
 'cd': 2.867301296020186e-05,
 'vbp': 8.112932013629726e-05,
 'nn': 7.841046309219502e-06,
 'vb': 3.9232610145552985e-05}

In [131]:
def Viterbi(sentence):
    V=[{}]
    for st in state_counts.keys():
#         print(sentence[0][0])
#         print(st)
        if st in emisson_matrix[sentence[0][0]].keys():
            emission_probability = emisson_matrix[sentence[0][0]][st]
        else:
            emission_probability = 0
        if st in transition_matrix['.'].keys():
            transition_probability = transition_matrix['.'][st]
        else:
            transition_probability = 0
#         print(transition_matrix['.'][st])
#         print(emisson_matrix[sentence[0][0]][st])
        V[0][st] = {"prob": emission_probability * transition_probability, "prev": None}
    
    for t in range(1, len(sentence)):
        V.append({})
        non_zero_initial_state = len(state_counts.keys())
        for st in state_counts.keys():
            keys = list(state_counts.keys())
            
            if st in transition_matrix[keys[0]]:
                initial_transition_probability = transition_matrix[keys[0]][st]
            else:
                initial_transition_probability = 0
                
            max_tr_prob = V[t - 1][keys[0]]["prob"] * initial_transition_probability
            prev_st_selected = keys[0]
            for prev_st in keys[1:]:
                #print(prev_st, st)
                if st in transition_matrix[prev_st]:
                    transition_probability = transition_matrix[prev_st][st]
                else:
                    transition_probability = 0
                #print(transition_matrix[prev_st][st])
                tr_prob = V[t - 1][prev_st]["prob"] * transition_probability
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st
                    
            if st in emisson_matrix[sentence[t][0]]:
                emission_probability = emisson_matrix[sentence[t][0]][st]
            else:
                emission_probability = 0
                
            max_prob = max_tr_prob * emission_probability
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
            if max_prob!=0:
                non_zero_initial_state -= 1
                
        #print(non_zero_initial_state)
        if non_zero_initial_state == len(state_counts.keys()):
            prob = list(V[t-1].values())
            #previous_probabilities = print(prob[0]['prob'])
            previous_probabilities = [value['prob'] for value in prob]
            previous_state = [value['prev'] for value in prob]
            previous_max = max(previous_probabilities)
            prev_max_st_index = previous_probabilities.index(previous_max)
            for st in V[t].keys():
                V[t][st] = {"prob": previous_max, "prev": previous_state[prev_max_st_index]}
        
    opt = []
    max_prob = 0.0
    best_st = None
    # Get most probable state and its backtrack
    #print(len(V))
    #print(sentence[15][0])
    #print(V[15].values())
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
    
    #print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)
    return opt

In [132]:
#print(len(cleaned_dev_corpus))
print(cleaned_dev_corpus[1958])
Tagged_POS = Viterbi(cleaned_dev_corpus[1958])
#print(cleaned_dev_corpus[66])
print(Tagged_POS)

[('mr.', 'nnp'), ('<unk>', 'nnp'), ('was', 'vbd'), ('hardly', 'rb'), ('<unk>', 'jjr'), ('to', 'to'), ('the', 'dt'), ('<unk>', 'nns'), ('of', 'in'), ('colleagues', 'nns'), ('michael', 'nnp'), ('<unk>', 'nnp'), ('(', '-lrb-'), ('``', '``'), ('appears', 'nns'), ('to', 'to'), ('be', 'vb'), ('a', 'dt'), ('<unk>', 'nn'), ("''", "''"), (')', '-rrb-'), (',', ','), ('will', 'nnp'), ('mcdonough', 'nnp'), ('(', '-lrb-'), ('``', '``'), ('looks', 'vbz'), ('as', 'in'), ('if', 'in'), ('he', 'prp'), ('drove', 'vbd'), ('for', 'in'), ('<unk>', 'nnp'), ('lincoln', 'nnp'), ("''", "''"), (')', '-rrb-'), ('or', 'cc'), ('<unk>', 'nnp'), ('english', 'nnp'), (',', ','), ('whose', 'wp$'), ('``', '``'), ('little', 'jj'), ('girl', 'nn'), ('now', 'rb'), ('<unk>', 'vbz'), ('<unk>', 'rb'), ('every', 'dt'), ('time', 'nn'), ('she', 'prp'), ('sees', 'vbz'), ('a', 'dt'), ('newspaper', 'nn'), ('.', '.'), ("''", "''")]
['nnp', 'nnp', 'vbd', 'rb', 'vbn', 'to', 'dt', 'nn', 'in', 'nns', 'nnp', 'nnp', '-lrb-', '``', 'vbz', 't

In [70]:
noErrors = cleaned_dev_corpus[:1696] + cleaned_dev_corpus[1697:1958] + cleaned_dev_corpus[1959:3321] + cleaned_dev_corpus[3322:]
#noErrors.pop(1696)
#noErrors.pop(1958)
#noErrors.pop(3321)

In [133]:
accuracy_model(cleaned_dev_corpus, Viterbi)

93.62585483222139